Exercise : MT Inversion\
Class    : Physics for Geosystems (M.Sc. Track GeoEnergy Engineering)

Department of Geoscience and Engineering\
Delft University and Technology


Created by La Ode Marzujriban Masfara © : l.o.m.masfara@tudelft.nl

In [6]:
#load all necessary packages
import numpy.matlib
import matplotlib.pyplot as plt
import numpy as np 
from decimal import Decimal
import warnings
warnings.filterwarnings("ignore")

# Problem setup

In this notebook, you are asked to do forward modeling for a specific MT and use the generated data to get the MT back using the full-waveform inversion technique. As discussed in the lecture, to model seismograms from specific MT, we must compute Green's functions beforehand to generate elementary seismograms. For this exercise, we assume that the elementary seismograms are known. The source location and receivers of these elementary seismograms are depicted in the figure below. The red star represents the source location, and each of the white triangles corresponds to the location of the receivers. In short, you are asked to do the following steps:

1. Create functions to convert from expansion coefficient a to M and vice versa. 
2. Define matrix M and use the function you create in step 1 to get a. Given a from your pre-defined M, generate the corresponding seismograms and plot them in the notebook.
3. Construct the required matrices for the forward problem (as discussed in the class).
4. Use the output of step 3 to invert for your pre-defined M.
5. Summarize what you have done in one cell of the notebook. Make sure you discuss the following: 
	
    a. Your choice of M (what does it represents geologically). 
	
    b. How is the amplitude of the first arrival look like on each receiver    (radiation pattern)

bonus points:
Generate random noise and use them to corrupt your forward modeled data and use it for the inversion. Discuss what you observe. 

6. Save your notebook and change the file name to your name and student number (NAME_STUDENTNUMBER.ipynb)


![title](studycase_001.png)

# Load data (time array and elementary seismogram)

Here we load the elementary seismogram given the estimated source location to each of the receivers. 


In [2]:
#load elementary seismogram & time array
elementaryseismo = np.load('elementaryseis.npy')
timearray = np.load('timearray.npy')

In [3]:
elementaryseismo.shape

(6934, 6, 3, 10)

Here is the detail of the elementaryseismo array:

- The first dimension corresponds to the number of samples in time (6934 sample)
- The second dimension corresponds to elementary seismogram (1-6)
- The third dimension is the recorded components (x,y,z)
- The fourth dimension is the receivers (see figure in the problem setup)

# 1 Functions AtoM & MtoA

The following is the transfer function from A to M:


$$
\mathbf{M}=\sum_{n=1}^{6} a_{n} \mathbf{M}^{n}=\left(\begin{array}{ccc}-a_{4}+a_{6} & a_{1} & a_{2} \\ a_{1} & -a_{5}+a_{6} & -a_{3} \\ a_{2} & -a_{3} & a_{4}+a_{5}+a_{6}\end{array}\right)
$$

# 2.1 Define your choice of M and compute A

Based on the expansion coefficients, displacements can be modeled using the relation below:
$$
u_{i}\left(\mathbf{x}_{r} ; \mathbf{x}_{a}, t\right)=\sum_{n=1}^{6} a_{n} S_{i}^{n}\left(\mathbf{x}_{r} ; \mathbf{x}_{a}, t\right)
$$

# 2.2 Plot the seismogram from the pre-defined M/A

# 3 Construct the forward problem matrices

# 4 solve the Inverse problem

Misfit function: 
$$
\chi(\mathbf{m})=\frac{1}{2}\left(\mathbf{d}^{\mathrm{obs}}-\mathbf{G m}\right)^{T}\left(\mathbf{d}^{\mathrm{obs}}-\mathbf{G m}\right)=\frac{1}{2} \sum_{i=1}^{N}\left(d_{i}^{\mathrm{obs}}-\sum_{j=1}^{n} G_{i j} m_{j}\right)^{2}
$$

the solution 
$$
\mathbf{m}^{\text {est }}=\left(\mathbf{G}^{T} \mathbf{G}\right)^{-1} \mathbf{G}^{T} \mathbf{d}^{\text {obs }}
$$

# 5 Summary
put your summary here...